In [69]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
load_dotenv()

True

In [70]:
from langchain.chat_models import ChatOpenAI

In [71]:
KEY=os.getenv("OPENAI_API_KEYS")

In [73]:
llm = ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)

In [75]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [76]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [77]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [78]:
quize_genration_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [79]:
quiz_chain = LLMChain(llm=llm, prompt=quize_genration_prompt, output_key="quiz", verbose=True)

In [80]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [81]:
quize_evaluation_prompt = PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
)

In [82]:
review_chain = LLMChain(llm=llm, prompt=quize_evaluation_prompt, output_key="review", verbose=True)

In [83]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text","number","subject","tone","response_json"],
    output_variables=["quiz","review"],
    verbose=True
)

In [99]:
file_path = r"C:\xampp\htdocs\GenAI\mcq_generator\data.txt"

In [100]:
file_path

'C:\\xampp\\htdocs\\GenAI\\mcq_generator\\data.txt'

In [101]:
with open(file_path,'r') as file:
    TEXT = file.read()

In [103]:
TEXT

'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field\'s methods.\n\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[7][8]\n\nFrom a 

In [104]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [105]:
NUMBER=5
SUBJECT="Machine Learning"
TONE="simple"

In [106]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text":TEXT,
            "number":NUMBER,
            "subject":SUBJECT,
            "tone":TONE,
            "response_json":json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field

In [107]:
response

{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field\'s methods.\n\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[7][8]\n

In [108]:
print(cb.total_tokens)
print(cb.prompt_tokens)
print(cb.completion_tokens)
print(cb.total_cost)

2379
1978
401
0.0037689999999999998


In [111]:
quiz = response.get('quiz')
quiz=json.loads(quiz)

In [112]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices":options,"Correct":correct})

In [113]:
quiz_table_data

[{'MCQ': 'What is machine learning?',
  'Choices': 'a: The study of artificial intelligence concerned with the development and study of statistical algorithms | b: The study of computer hardware | c: The study of human cognition | d: The study of quantum mechanics',
  'Correct': 'a'},
 {'MCQ': "Who coined the term 'machine learning'?",
  'Choices': 'a: Alan Turing | b: Arthur Samuel | c: Donald Hebb | d: Tom M. Mitchell',
  'Correct': 'b'},
 {'MCQ': 'What was the earliest machine learning model introduced in the 1950s?',
  'Choices': 'a: Program that calculated winning chance in chess | b: Program that calculated winning chance in checkers | c: Program that calculated winning chance in tic-tac-toe | d: Program that calculated winning chance in poker',
  'Correct': 'b'},
 {'MCQ': 'What is the main objective of modern-day machine learning?',
  'Choices': 'a: To classify data based on models developed | b: To predict future outcomes based on models | c: Both a and b | d: None of the above

In [114]:
quiz=pd.DataFrame(quiz_table_data)

In [115]:
quiz

,MCQ,Choices,Correct
0,What is machine learning?,a: The study of artificial intelligence concer...,a
1,Who coined the term 'machine learning'?,a: Alan Turing | b: Arthur Samuel | c: Donald ...,b
2,What was the earliest machine learning model i...,a: Program that calculated winning chance in c...,b
3,What is the main objective of modern-day machi...,a: To classify data based on models developed ...,c
4,What field did machine learning grow out of as...,a: Computer graphics | b: Artificial intellige...,b


In [116]:
quiz.to_csv("machinelearning.csv",index=False)

In [117]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'05_22_2024_14_22_15'